# <font><center>**COMP5328 Assignment 2**</center></font>
## <center>**Group 42**</center>
#### <center>Group Member</center>
<div style="text-align: center;">
  <table style="display: inline-block;">
    <tr>
      <th>SID</th>
      <th>Unikey</th>
      <th>Name</th>
    </tr>
    <tr>
      <td>520562127</td>
      <td>cser6317</td>
      <td>Jeremiah Ser</td>
    </tr>
    <tr>
      <td>510640657</td>
      <td>jhua2634</td>
      <td>Jian Huang</td>
    </tr>
    <tr>
      <td>520305074</td>
      <td>kche3315</td>
      <td>Kehao Chen</td>
    </tr>
  </table>
</div>




### Code instruction

1. All the packages used in this study are listed in "Import packages" section below. If any packages are not installed, please use the command **!pip install package_name** to install them. Please update the package version if needed.

2. Run the code cells sequentially, following the instructions and comments provided within the notebook.

# Import packages

In [1]:
import numpy as np
from tabulate import tabulate
from LoadDataset import *
from LossFunction import *
from Models import *
from TrainTest import *
from TrainTestMixup import *
from TrainTestVMNet import *

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import warnings
warnings.simplefilter('ignore')

## Experiment Set up

In [2]:
# instantiate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Set random seed for reproducibility
np.random.seed(42)

# Set Train and Validation split 
train_ratio = 0.8

# Set number of epoch 
epoch = 20

# Number of training
sample_range = 10

# Transition matrix for FashionMNIST datasets
given_matrix = np.array(
    [
        [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]], # FashionMNIST0.5
        [[0.4, 0.3, 0.3], [0.3, 0.4, 0.3], [0.3, 0.3, 0.4]], # FashionMNIST0.6
    ]
)


## Baseline LeNet-5 model testing

In [3]:
Result_avg, Result_sd = [], []
for path in [
    "../data/FashionMNIST0.5.npz",
    "../data/FashionMNIST0.6.npz",
    "../data/CIFAR.npz",
]:
    test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list = [], [], [], [], []
    for random_seed in range(sample_range):
        # load data: FashionMNIST0.5.npz, FashionMNIST0.6.npz, CIFAR.npz
        train_loader, val_loader, test_loader = load_data(
            path=path, batch_size=256, train_ratio=train_ratio, random_seed=random_seed
        )

        # model
        if path == "../data/CIFAR.npz":
            model = LeNet5_32().to(device)
        else:
            model = LeNet5_28().to(device)

        # optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        # loss function
        criterion = nn.CrossEntropyLoss(None)

        # train
        print("*" * 50, f"{path[8:]} trained with random seed {random_seed}", "*" * 50)
        train_metrics, val_metrics = train(
            model,
            optimizer,
            criterion,
            train_loader,
            val_loader,
            epochs=epoch,
            device=device,
        )

        # test
        test_loss, test_acc, test_f1, test_precision, test_recall = test(
            model, criterion, test_loader, device=device
        )
        test_loss_list.append(round(test_loss, 4))
        test_acc_list.append(round(test_acc.item(), 4))
        test_f1_list.append(round(test_f1.item(), 4))
        test_precision_list.append(round(test_precision.item(), 4))
        test_recall_list.append(round(test_recall.item(), 4))

    Result_avg.append(
        list(map(np.average, [test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list]))
    )
    Result_sd.append(list(map(np.std, [test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list])))


************************************************** FashionMNIST0.5.npz trained with random seed 0 **************************************************
Epoch 5/20 - Train Loss: 1.0353, Train Accuracy: 0.4907, Train F1: 0.4907, Train Precision: 0.4908, Train Recall: 0.4907, Val Loss: 1.0589, Val Acc: 0.4725, Val F1: 0.4723, Val Precision: 0.4742, Val Recall: 0.4725
Epoch 10/20 - Train Loss: 0.9910, Train Accuracy: 0.5161, Train F1: 0.5159, Train Precision: 0.5169, Train Recall: 0.5161, Val Loss: 1.0928, Val Acc: 0.4650, Val F1: 0.4651, Val Precision: 0.4668, Val Recall: 0.4650
Epoch 15/20 - Train Loss: 0.8568, Train Accuracy: 0.5881, Train F1: 0.5873, Train Precision: 0.5937, Train Recall: 0.5881, Val Loss: 1.2251, Val Acc: 0.4425, Val F1: 0.4405, Val Precision: 0.4416, Val Recall: 0.4425
Epoch 20/20 - Train Loss: 0.6893, Train Accuracy: 0.6786, Train F1: 0.6783, Train Precision: 0.6870, Train Recall: 0.6786, Val Loss: 1.7728, Val Acc: 0.4286, Val F1: 0.4265, Val Precision: 0.4272, Val Rec

## Volume minimization network (VolMinNet)

### Initialize transition matrix for noise rate estimation

In [4]:
def initializeT():
    # Initialize a C x C matrix for T_hat
    C = 3  # Number of classes
    T_init = torch.zeros(C, C, requires_grad=False)  # Initialize with zeros

    # Make the matrix diagonally dominant
    T_init.fill_diagonal_(1.0)

    # Initialize off-diagonal elements with small random values and apply sigmoid
    for i in range(C):
        for j in range(C):
            if i != j:
                T_init[i, j] = torch.sigmoid(
                    torch.rand(1) * 0.1
                )  # Initialize with small random values and apply sigmoid

    # Normalize the columns
    T_init = normalize_columns(T_init)

    # Convert to a leaf tensor
    T_hat = (
        T_init.clone().detach().requires_grad_(True)
    )  # Now T_hat is a leaf tensor with requires_grad=True

    # Move T_hat to the device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    with torch.no_grad():
        T_hat = T_hat.clone().to(device)
    T_hat.requires_grad_(True)
    return T_hat


### VolMinNet Testing

In [5]:
estimated_matrices, Result_avg_vmn, Result_sd_vmn = [], [], []
for path in [
    "../data/FashionMNIST0.5.npz",
    "../data/FashionMNIST0.6.npz",
    "../data/CIFAR.npz",
]:
    test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list, T_hat_list = [], [], [], [], [], []
    
    for random_seed in range(sample_range):
        T_hat = initializeT()
        # load data, we use one-hot encoding for mixup
        train_loader, val_loader, test_loader = load_data(
            path="../data/FashionMNIST0.5.npz",
            batch_size=256,
            train_ratio=train_ratio,
            random_seed=0,
            to_onehot=False,
        )

        # model
        if path == "../data/CIFAR.npz":
            model = LeNet5_28().to(device)
        else:
            model = LeNet5_28().to(device)

        # Classifier optimizer
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        # Transition matrix optimizer
        optimizer_T = optim.Adam([T_hat], lr=1e-4)

        # loss function
        criterion = nn.NLLLoss()
        # Train
        print("*" * 50, f"{path[8:]} trained with random seed {random_seed}", "*" * 50)
        train_metrics, val_metrics = train_vmn(
            model,
            optimizer,
            optimizer_T,
            criterion,
            T_hat,
            train_loader,
            val_loader,
            epochs=epoch,
            device=device,
            lambda_param=1e-3,
        )
        # Test
        test_loss, test_acc, test_f1, test_precision, test_recall = test_vmn(
            model, criterion, T_hat, test_loader, device=device
        )
        test_loss_list.append(round(test_loss, 4))
        test_acc_list.append(round(test_acc.item(), 4))
        test_f1_list.append(round(test_f1.item(), 4))
        test_precision_list.append(round(test_precision.item(), 4))
        test_recall_list.append(round(test_recall.item(), 4))
        T_hat_list.append(np.round(T_hat.requires_grad_(False).cpu().numpy(), 4))
    estimated_matrices.append(np.sum(T_hat_list, axis=0)/10)

    Result_avg_vmn.append(
        list(map(np.average, [test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list]))
    )
    Result_sd_vmn.append(
        list(map(np.std, [test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list]))
    )


************************************************** FashionMNIST0.5.npz trained with random seed 0 **************************************************
Epoch 5 - T_hat: 
 [[0.48118064 0.25842687 0.2609084 ]
 [0.25894326 0.4793454  0.2569248 ]
 [0.259876   0.26222777 0.4821669 ]]
Epoch 5/20 - Train Loss: 1.0511, Train Accuracy: 0.4818, Train F1: 0.4816, Train Precision: 0.4822, Train Recall: 0.4818, Val Loss: 1.0549, Val Acc: 0.4728, Val F1: 0.4730, Val Precision: 0.4740, Val Recall: 0.4728
Epoch 10 - T_hat: 
 [[0.47084716 0.26488483 0.26620883]
 [0.26405388 0.46886015 0.26308945]
 [0.26509896 0.266255   0.47070172]]
Epoch 10/20 - Train Loss: 1.0481, Train Accuracy: 0.4910, Train F1: 0.4909, Train Precision: 0.4911, Train Recall: 0.4910, Val Loss: 1.0581, Val Acc: 0.4717, Val F1: 0.4713, Val Precision: 0.4740, Val Recall: 0.4717
Epoch 15 - T_hat: 
 [[0.46137258 0.2702952  0.27075323]
 [0.2686706  0.45949227 0.2682233 ]
 [0.26995683 0.2702126  0.46102348]]
Epoch 15/20 - Train Loss: 1.0467, 

### Evaluate noise rate (transition matrix) estimation by VioMinNet

In [6]:
for i in range(2):
    print(
        "True transition matrix:\n\n",
        given_matrix[i],
        "\n",
        "Estimated transition matrix:\n\n",
        estimated_matrices[i],
        "\n",
        "MSE :",
        np.mean((estimated_matrices[i] - given_matrix[i]) ** 2),
        "\n"
    )
given_matrix = np.vstack((given_matrix, [estimated_matrices[2]]))
print("Transition matrix estimation for CIFAR dataset:\n\n", estimated_matrices[2])


True transition matrix:

 [[0.5 0.2 0.3]
 [0.3 0.5 0.2]
 [0.2 0.3 0.5]] 
 Estimated transition matrix:

 [[0.45337    0.27601    0.27243   ]
 [0.27226    0.45422006 0.27530998]
 [0.2744     0.26981    0.45224   ]] 
 MSE : 0.0028863000204185866 

True transition matrix:

 [[0.4 0.3 0.3]
 [0.3 0.4 0.3]
 [0.3 0.3 0.4]] 
 Estimated transition matrix:

 [[0.45313    0.27641    0.27213   ]
 [0.27264    0.45373002 0.27681   ]
 [0.27423    0.26986998 0.45106   ]] 
 MSE : 0.0013898129614453645 

Transition matrix estimation for CIFAR dataset:

 [[0.45255    0.27629    0.27196997]
 [0.2739     0.45422    0.27792   ]
 [0.27354997 0.26946998 0.4501    ]]


## Forward learning using known transition matrix

In [7]:
# Train
Result_avg_fl, Result_sd_fl = [], []
for path in [
    "../data/FashionMNIST0.5.npz",
    "../data/FashionMNIST0.6.npz",
    "../data/CIFAR.npz",
]:
    # The matrices are known
    t = None
    if path == "../data/FashionMNIST0.5.npz":
        t = given_matrix[0]
    if path == "../data/FashionMNIST0.6.npz":
        t = given_matrix[1]
    if path == "../data/CIFAR.npz":
        t = given_matrix[2]
    test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list = [], [], [], [], []
    for random_seed in range(sample_range):
        # load data: FashionMNIST0.5.npz, FashionMNIST0.6.npz, CIFAR.npz
        train_loader, val_loader, test_loader = load_data(
            path=path, batch_size=256, train_ratio=train_ratio, random_seed=random_seed
        )

        # model
        if path == "../data/CIFAR.npz":
            model = LeNet5_32().to(device)
        else:
            model = LeNet5_28().to(device)

        # optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.0001)
        # loss function
        criterion = NoiseAwareCELoss(t, device=device)

        # train
        print("*" * 50, f"{path[8:]} trained with random seed {random_seed}", "*" * 50)
        train_metrics, val_metrics = train(
            model,
            optimizer,
            criterion,
            train_loader,
            val_loader,
            epochs=epoch,
            device=device,
        )

        # test
        test_loss, test_acc, test_f1, test_precision, test_recall = test(
            model, criterion, test_loader, device=device
        )
        test_loss_list.append(round(test_loss, 4))
        test_acc_list.append(round(test_acc.item(), 4))
        test_f1_list.append(round(test_f1.item(), 4))
        test_precision_list.append(round(test_precision.item(), 4))
        test_recall_list.append(round(test_recall.item(), 4))
        

    Result_avg_fl.append(
        list(map(np.average, [test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list]))
    )
    Result_sd_fl.append(
        list(map(np.std, [test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list]))
    )


************************************************** FashionMNIST0.5.npz trained with random seed 0 **************************************************
Epoch 5/20 - Train Loss: 1.0406, Train Accuracy: 0.4806, Train F1: 0.4806, Train Precision: 0.4807, Train Recall: 0.4806, Val Loss: 1.0445, Val Acc: 0.4775, Val F1: 0.4776, Val Precision: 0.4778, Val Recall: 0.4775
Epoch 10/20 - Train Loss: 1.0361, Train Accuracy: 0.4897, Train F1: 0.4897, Train Precision: 0.4897, Train Recall: 0.4897, Val Loss: 1.0439, Val Acc: 0.4781, Val F1: 0.4780, Val Precision: 0.4796, Val Recall: 0.4781
Epoch 15/20 - Train Loss: 1.0279, Train Accuracy: 0.5001, Train F1: 0.5000, Train Precision: 0.5001, Train Recall: 0.5001, Val Loss: 1.0427, Val Acc: 0.4803, Val F1: 0.4803, Val Precision: 0.4804, Val Recall: 0.4803
Epoch 20/20 - Train Loss: 1.0230, Train Accuracy: 0.5076, Train F1: 0.5076, Train Precision: 0.5077, Train Recall: 0.5076, Val Loss: 1.0450, Val Acc: 0.4753, Val F1: 0.4754, Val Precision: 0.4757, Val Rec

## Mix-up augmentation testing

In [8]:
# Train
Result_avg_mix, Result_sd_mix = [], []
for path in [
    "../data/FashionMNIST0.5.npz",
    "../data/FashionMNIST0.6.npz",
    "../data/CIFAR.npz",
]:
    test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list = [], [], [], [], []
    
    for random_seed in range(sample_range):
        # load data: FashionMNIST0.5.npz, FashionMNIST0.6.npz
        train_loader, val_loader, test_loader = load_data(
            path=path,
            batch_size=256,
            train_ratio=train_ratio,
            random_seed=random_seed,
            to_onehot=True,
        )

        # model
        if path == "../data/CIFAR.npz":
            model = LeNet5_32().to(device)
        else:
            model = LeNet5_28().to(device)

        # optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        # loss function
        criterion = nn.CrossEntropyLoss()

        # train
        print("*" * 50, f"{path[8:]} trained with random seed {random_seed}", "*" * 50)
        train_metrics, val_metrics = train_mixup(
            model,
            optimizer,
            criterion,
            train_loader,
            val_loader,
            epochs=epoch,
            device=device,
            alpha=2,
        )
        # test
        test_metrics = test_mixup(model, criterion, test_loader, device=device)
        test_loss_list.append(round(test_metrics["loss"], 4))
        test_acc_list.append(round(test_metrics["acc"], 4))
        test_f1_list.append(round(test_metrics["f1"], 4))
        test_precision_list.append(round(test_metrics["pre"], 4))
        test_recall_list.append(round(test_metrics["re"], 4))

    Result_avg_mix.append(
        list(map(np.average, [test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list]))
    )
    Result_sd_mix.append(
        list(map(np.std, [test_loss_list, test_acc_list, test_f1_list, test_precision_list, test_recall_list]))
    )


************************************************** FashionMNIST0.5.npz trained with random seed 0 **************************************************
Epoch 5/20 - Train Loss: 1.0697, Train Accuracy: 0.4573, Train F1: 0.4571, Train Precision: 0.4572, Train Recall: 0.4573, Val Loss: 1.0552, Val Acc: 0.4719, Val F1: 0.4718, Val Precision: 0.4725, Val Recall: 0.4719
Epoch 10/20 - Train Loss: 1.0624, Train Accuracy: 0.4683, Train F1: 0.4683, Train Precision: 0.4682, Train Recall: 0.4683, Val Loss: 1.0617, Val Acc: 0.4722, Val F1: 0.4721, Val Precision: 0.4732, Val Recall: 0.4722
Epoch 15/20 - Train Loss: 1.0609, Train Accuracy: 0.4684, Train F1: 0.4681, Train Precision: 0.4685, Train Recall: 0.4684, Val Loss: 1.0612, Val Acc: 0.4728, Val F1: 0.4728, Val Precision: 0.4733, Val Recall: 0.4728
Epoch 20/20 - Train Loss: 1.0596, Train Accuracy: 0.4688, Train F1: 0.4687, Train Precision: 0.4690, Train Recall: 0.4688, Val Loss: 1.0673, Val Acc: 0.4742, Val F1: 0.4742, Val Precision: 0.4753, Val Rec

## Experiment Result

### Noise rate (transitioin matrix) estimation for unknwon flip rate testing (CIFAR dataset)

In [9]:
print("Evaluated matrix of CIFAR dataset:\n\n", estimated_matrices[2])

Evaluated matrix of CIFAR dataset:

 [[0.45255    0.27629    0.27196997]
 [0.2739     0.45422    0.27792   ]
 [0.27354997 0.26946998 0.4501    ]]


### Classification performance

In [10]:
data = [
        ["FashionMNIST0.5", "Baseline", Result_avg[0][0],Result_sd[0][0],Result_avg[0][1],Result_sd[0][1],Result_avg[0][2],Result_sd[0][2], 
         Result_avg[0][3],Result_sd[0][3], Result_avg[0][4], Result_sd[0][4]],

        ["FashionMNIST0.5", "Forward Learning", Result_avg_fl[0][0],Result_sd_fl[0][0],Result_avg_fl[0][1],Result_sd_fl[0][1],Result_avg_fl[0][2],Result_sd_fl[0][2],
         Result_avg_fl[0][3],Result_sd_fl[0][3],Result_avg_fl[0][4],Result_sd_fl[0][4]],

        ["FashionMNIST0.5", "Mixup", Result_avg_mix[0][0],Result_sd_mix[0][0],Result_avg_mix[0][1],Result_sd_mix[0][1],Result_avg_mix[0][2],Result_sd_mix[0][2],
         Result_avg_mix[0][3],Result_sd_mix[0][3],Result_avg_mix[0][4],Result_sd_mix[0][4]],

        ["FashionMNIST0.5", "VMNet", Result_avg_vmn[0][0],Result_sd_vmn[0][0],Result_avg_vmn[0][1],Result_sd_vmn[0][1],Result_avg_vmn[0][2],Result_sd_vmn[0][2],
         Result_avg_vmn[0][3],Result_sd_vmn[0][3],Result_avg_vmn[0][4],Result_sd_vmn[0][4]],

        ["FashionMNIST0.6", "Baseline", Result_avg[1][0],Result_sd[1][0],Result_avg[1][1],Result_sd[1][1],Result_avg[1][2],Result_sd[1][2],
         Result_avg[1][3],Result_sd[1][3], Result_avg[1][4],Result_sd[1][4]],

        ["FashionMNIST0.6", "Forward Learning", Result_avg_fl[1][0],Result_sd_fl[1][0],Result_avg_fl[1][1],Result_sd_fl[1][1],Result_avg_fl[1][2],Result_sd_fl[1][2],
         Result_avg_fl[1][3],Result_sd_fl[1][3], Result_avg_fl[1][4],Result_sd_fl[1][4]], 
        
        ["FashionMNIST0.6", "Mixup", Result_avg_mix[1][0],Result_sd_mix[1][0],Result_avg_mix[1][1],Result_sd_mix[1][1],Result_avg_mix[1][2],Result_sd_mix[1][2],
         Result_avg_mix[1][3],Result_sd_mix[1][3], Result_avg_mix[1][4],Result_sd_mix[1][4]], 

        ["FashionMNIST0.6", "VMNet", Result_avg_vmn[1][0],Result_sd_vmn[1][0],Result_avg_vmn[1][1],Result_sd_vmn[1][1],Result_avg_vmn[1][2],Result_sd_vmn[1][2],
         Result_avg_vmn[1][3],Result_sd_vmn[1][3],Result_avg_vmn[1][4],Result_sd_vmn[1][4]],

        ["CIFAR", "Baseline", Result_avg[2][0],Result_sd[2][0],Result_avg[2][1],Result_sd[2][1],Result_avg[2][2],Result_sd[2][2],
         Result_avg[2][3],Result_sd[2][3], Result_avg[2][4],Result_sd[2][4]],

        ["CIFAR", "Forward Learning", Result_avg_fl[2][0],Result_sd_fl[2][0],Result_avg_fl[2][1],Result_sd_fl[2][1],Result_avg_fl[2][2],Result_sd_fl[2][2],
         Result_avg_fl[2][3],Result_sd_fl[2][3], Result_avg_fl[2][4],Result_sd_fl[2][4]],

        ["CIFAR", "Mixup", Result_avg_mix[2][0],Result_sd_mix[2][0],Result_avg_mix[2][1],Result_sd_mix[2][1],Result_avg_mix[2][2],Result_sd_mix[2][2],
         Result_avg_mix[2][3],Result_sd_mix[2][3], Result_avg_mix[2][4],Result_sd_mix[2][4]],

        ["CIFAR", "VMNet", Result_avg_vmn[2][0],Result_sd_vmn[2][0],Result_avg_vmn[2][1],Result_sd_vmn[2][1],Result_avg_vmn[2][2],Result_sd_vmn[2][2],
         Result_avg_vmn[2][3],Result_sd_vmn[2][3], Result_avg_vmn[2][4],Result_sd_vmn[2][4]],
        ]

headers = ["Dataset", "Classifier", "Loss(avg)", "Loss(sd)", "Acc(avg)", "Acc(sd)", "F1(avg)","F1(sd)", "Precision(avg)", "Precision(sd)", 
           "Recall(avg)", "Recall(sd)"]

print(tabulate(data, headers, tablefmt="fancy_grid"))

╒═════════════════╤══════════════════╤═════════════╤════════════╤════════════╤════════════╤═══════════╤════════════╤══════════════════╤═════════════════╤═══════════════╤══════════════╕
│ Dataset         │ Classifier       │   Loss(avg) │   Loss(sd) │   Acc(avg) │    Acc(sd) │   F1(avg) │     F1(sd) │   Precision(avg) │   Precision(sd) │   Recall(avg) │   Recall(sd) │
╞═════════════════╪══════════════════╪═════════════╪════════════╪════════════╪════════════╪═══════════╪════════════╪══════════════════╪═════════════════╪═══════════════╪══════════════╡
│ FashionMNIST0.5 │ Baseline         │     1.19263 │ 0.233822   │    0.65027 │ 0.0623055  │   0.64764 │ 0.0625777  │          0.65048 │      0.0626232  │       0.65027 │   0.0623055  │
├─────────────────┼──────────────────┼─────────────┼────────────┼────────────┼────────────┼───────────┼────────────┼──────────────────┼─────────────────┼───────────────┼──────────────┤
│ FashionMNIST0.5 │ Forward Learning │     0.75632 │ 0.0065804  │    0.9251